# 05.概率、随机性与不稳定性

前面已经讲到，LLM的本质是预测下一个Token，每次返回的是一个概率分布，所以一次回答是多次概率采样的连锁结果。

## 一、为什么同一个问题多次询问会得到不同答案

假设输入是：“天空是___”
模型计算出的概率可能是：

* **蓝色的**：0.7
* **阴沉的**：0.15
* **广阔的**：0.1
* **红色的**：0.05

如果模型每次都只选概率最高的那一个（贪婪搜索），那么它确实会永远给出相同的答案。但在实际应用中，为了让语言更自然、更有创造力，会引入**随机采样**。

需要注意的是这里的“不同”并不等同于“错误”。在向量空间中，“正确答案”往往不是唯一的**，尤其是解释类、总结类、推理路径类的问题。LLM的目标不是“一致性”，而是**在统计意义上像人类写的文本**.

### 实战：同一问题，多次采样

In [1]:
# 加载环境变量

import os
from dotenv import load_dotenv

load_dotenv()

assert os.getenv("OPENAI_API_KEY"), "请先配置 OPENAI_API_KEY"
print("✅ API Key loaded")

✅ API Key loaded


In [3]:
from openai import OpenAI

client = OpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

def ask_llm(prompt, temperature=0.7, top_p=1.0):
    response = client.chat.completions.create(
        model="qwen-flash",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content

In [4]:
prompt = "请用三句话解释什么是大语言模型。"

for i in range(5):
    print(f"\n--- 第 {i+1} 次 ---")
    print(ask_llm(prompt, temperature=0.7))


--- 第 1 次 ---
大语言模型是一种基于海量文本数据训练的深度学习模型，能够理解并生成人类语言。它通过学习语言中的统计规律，具备回答问题、撰写文章、翻译语句等自然语言处理能力。其“大”主要体现在参数量巨大，从而拥有更强的泛化和上下文理解能力。

--- 第 2 次 ---
大语言模型是一种基于大量文本数据训练的深度学习模型，能够理解、生成和推理自然语言。它通过学习语言中的统计规律，实现文本续写、问答、翻译等多种语言任务。因其庞大的参数量和强大的泛化能力，大语言模型在各类自然语言处理应用中表现出色。

--- 第 3 次 ---
大语言模型是一种基于海量文本数据训练的深度学习模型，能够理解并生成接近人类水平的自然语言。它通过预测下一个词或补全文本，在对话、写作、翻译等任务中表现出强大的语言能力。其核心优势在于“预训练+微调”的模式，能适应多种不同的应用场景。

--- 第 4 次 ---
大语言模型是一种基于海量文本训练的深度学习模型，能够理解并生成人类语言。它通过分析大量语料库中的语言模式，具备强大的文本生成、问答、翻译和摘要等能力。尽管其表现接近人类水平，但模型本身并无意识或理解力，仅依赖统计规律进行预测。

--- 第 5 次 ---
大语言模型是一种基于海量文本数据训练的深度学习模型，能够理解和生成人类语言。它通过捕捉语言中的统计规律，实现文本生成、问答、翻译等多种自然语言处理任务。其“大”主要体现在参数量巨大，具备强大的泛化能力和上下文理解能力。


**结果分析**：从上述结果可以看到，虽然返回结果的核心意思一致，但每次的侧重点都不一样。

既然模型输出的多样性并不是随机的“噪声”，而是由生成策略和控制参数决定的。那么，我们该如何控制这种随机性的“度”？

答案是通过调整 temperature、top-k 或 top-p 等采样参数，我们可以控制模型在生成时选择高概率词的倾向，或者允许它尝试概率较低但更有创意的词汇。理解这些参数如何影响生成过程，是解释为什么同一个问题会产生不同回答的核心，也是学习如何平衡一致性与创造性的基础。

## 二、temperature等参数

1. **Temperature（温度）**

   * 控制概率分布的平滑程度。
   * **低温度（如 0.1–0.3）**：模型更“保守”，高概率词更容易被选中 → 输出更稳定、一致性高。
   * **高温度（如 0.7–1.0）**：概率分布更平坦，低概率词更有机会被采样 → 输出更丰富、创造性更强。

2. **Top-k 采样**

   * 仅从概率排名前 k 的候选词中进行采样。
   * 限制范围可以避免模型选择极低概率的词，提高生成合理性。
   * k 越小 → 输出越确定，k 越大 → 输出更多样化。

3. **Top-p 采样（又称 nucleus 采样）**

   * 选择累积概率达到 p 的候选词集合进行采样。
   * 比 Top-k 更灵活，因为候选数量会根据概率分布自动调整。
   * p 越小 → 输出越保守，p 越大 → 输出更自由、多样。


其中Temperature和Top-k比较好理解，而Top-p有点复杂，这里补充说明下：
>Nucleus Sampling 的工作原理：
>1. 计算累积概率：
>  - 首先，对模型输出的概率分布进行排序，并计算词汇的累积概率。
>  - 然后，根据预先设定的阈值 p，确定一个最小集合，使得这个集合内的词汇的累积概率总和不超过 p。
>2. 动态选择词汇：
>  - 在每个时间步，根据计算出的累积概率和阈值 p，动态地选择词汇。
>  - 如果累积概率中的某个词汇使得总概率超过 p，则剔除该词汇并重新分配剩余概率给其他词汇。
>3. 控制多样性：
>  - 通过调节阈值 p 的大小，可以控制生成文本的多样性。较小的 p 值会导致选择的词汇更加集中，而较大的 p 值会增加选择的词汇范围。
>示例说明：
>假设我们有一个词汇表，包含以下单词及其对应概率：
>- A: 0.3
>- B: 0.2
>- C: 0.15
>- D: 0.1
>- E: 0.1
>- F: 0.05
>- G: 0.05
>- H: 0.05
>假设我们设置阈值 p=0.6。
>1. 计算累积概率：
>  - 累积概率序列为：0.3, 0.5, 0.65, 0.75, 0.85, 0.9, 0.95, 1.0。
>2. 动态选择词汇：
>  - 当累积概率超过 p=0.6 时停止选择，例如选择前三个单词 A, B, C。
>  - 如果某单词的累积概率使得总和超过 p，则该单词会被剔除，例如单词 D。
>通过 Nucleus Sampling，模型可以在保持一定多样性的同时，根据上下文动态地选择词汇，使生成文本更加丰富和有趣。通过调节阈值 p，可以控制生成文本的多样性和可控性，使其更>符合期望。

### 实战：Temperatrue实现

In [5]:
temperatures = [0.0, 0.3, 0.7, 1.0]

prompt = "请解释为什么 LLM 天生不稳定。"

for t in temperatures:
    print(f"\n=== temperature = {t} ===")
    print(ask_llm(prompt, temperature=t))


=== temperature = 0.0 ===
大型语言模型（Large Language Models, LLMs）“天生不稳定”这一说法，实际上并非指它们在技术上无法稳定运行，而是指它们在**行为、输出和推理过程中的不可预测性与不一致性**，这种现象源于其设计原理和训练机制。以下是几个关键原因，解释为什么 LLM 被认为“天生不稳定”：

---

### 1. **基于概率的生成机制**
LLM 的核心是**概率建模**：它们通过学习海量文本数据中词语之间的统计关联，预测下一个词的概率分布。这意味着：
- 模型不会“知道”答案，而是“猜测”最可能的词序列。
- 即使输入完全相同，由于采样策略（如 temperature、top-k、top-p）不同，输出也可能变化。
- 这种随机性使得模型输出在某些情况下看似“不稳定”。

👉 **举例**：问同一个问题两次，模型可能给出两个风格或内容略有差异的回答。

---

### 2. **缺乏真正的理解与逻辑推理能力**
LLM 不具备人类意义上的“理解”或“意识”。它们只是根据上下文和模式匹配生成文本，因此：
- 对复杂、抽象或矛盾的问题，可能产生自相矛盾的回应。
- 在逻辑推理任务中容易“自圆其说”，但结论可能是错的。
- 面对未见过的场景时，容易“胡编乱造”（hallucination）。

👉 这种“看似合理但实际错误”的输出，让人感觉模型“不可靠”或“不稳定”。

---

### 3. **对输入微小变化敏感（幻觉与漂移）**
即使输入仅有一个字的变化，模型的输出可能显著不同。例如：
- “请告诉我如何做蛋糕” → 详细食谱
- “请告诉我如何做‘蛋糕’” → 可能误解为“蛋糕”是一种宗教仪式

这种对输入扰动的高度敏感，使得模型行为难以预测，表现为“不稳定”。

---

### 4. **训练数据的噪声与偏见**
LLM 训练数据来自互联网，包含大量不一致、错误、争议或偏见信息。这导致：
- 模型学到的“常识”可能是矛盾的。
- 对同一主题可能有多个“合理”但互相冲突的版本。
- 模型在面对模糊或边界情况时，选择哪个“合理”版本是不确定的。

👉 结果就是：同样的问题，在不同时间或不同设置下，可能得到不同的回答。

---

### 5. **没有明确的目标函数或一致性约束**
与传统程序不同，

你可以多运行几次对比下不同temperature下的返回表现。

值得注意的是，由于由于 LLM 底层采样具有随机性，即使 Temperature 设置得再低，有时也难保两次输出完全一致。所以像阿里云大模型的API中加入了Seed (随机数种子)参数，如果你在两次请求中输入了相同的 Prompt、相同的 Temperature 以及相同的 Seed，那么模型大概率会给你生成完全相同的答案。

## 三、LLM 擅长什么，不擅长什么

基于LLM 的底层是概率分布和模式识别，这决定了它在处理**非结构化、有灵活度**的任务时表现惊人，但也有其短板。

### LLM 擅长的：

* **模式匹配**：识别文本中的结构、风格或隐含的逻辑。比如给模型一段你的写作风格，让它模仿。它能迅速捕捉你常用的句式、语气。
* **语言生成*：将抽象的意图转化为流畅、多样的表达。比如要求“将这段技术文档改写成给 5 岁小孩听的故事”，它能完美完成语义的跨域平移。
* **近似推理**：处理没有标准答案、需要常识和直觉的推理。比如判断一段对话中用户的潜在情绪。模型不是靠规则判断，而是靠“感受”词汇间的语义距离。
* **模糊问题**：理解不完整、不精确的指令。比如你输入“那个讲外星人到地球结果发现人类很落后的老科幻电影叫什么？”（即使信息模糊，它也能匹配到《地球停转之日》）。

### LLM 不擅长的：

LLM 的致命弱点源于它**“不查表、不计算”**。它在面对需要绝对精确性的任务（比如逻辑与事实）时极易翻车。

* **精确计算**：模型是在预测数字的“长相”，而不是在执行算术逻辑。比如询问计算结果时，它可能会给出一个“看起来很像正确答案”的数字，但由于进位错误，末尾几位往往是错的。
* **事实校验 **：它优先保证句子的“通顺”，而非“真实”。这就是所谓的“幻觉”。比如询问“2023 年诺贝尔文学奖获得者写的关于赛博朋克的小说叫什么？”即使该作者没写过，模型也可能编造出一个书名。
* **长期一致性**：受限于上下文窗口，生成内容过长时，模型会忘记前文的设定。比如在写长篇小说时，第 1 章设定主角是左撇子，第 50 章可能就变成了右手。
* **边界条件 (Edge Cases)***：对于罕见、极端的逻辑问题，由于训练数据覆盖不足，模型常表现出“过度自信的错误”。
* **自我纠错 (Self-Correction)**：模型往往会顺着自己的错话继续往下编，很难在生成过程中突然意识到“我刚才说错了”。比如当模型算错题时，如果你问“你确定吗？”，它常会道歉并给出**另一个**同样错误的答案，而不是重新审视逻辑。
